In [1]:
import pandas as pd
import numpy as np
# from myUtils import pickle_load, symb_perf_stats_vectorized, symb_perf_stats_vectorized_v1
from myUtils import pickle_load, symb_perf_stats_vectorized_v2

file_close = 'df_close_clean'
path_dir = "C:/Users/ping/MyDrive/stocks/yfinance"
path_data_dump = path_dir + "/VSCode_dump/"

df_c = pickle_load(path_data_dump, file_close)
df_c = df_c[-252::]
format1 = 'df_c({}):\n{}\n{}'.format(len(df_c), df_c.head(3), df_c.tail(3))
# print(format1)

In [2]:
# # %%timeit -n 20 -r 17
# symbols, period_yr, drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = \
#     symb_perf_stats_vectorized_v2(df_c)

In [3]:
def perf_eval(df_close):
  '''
  df_close is a dataframe with date index, columns of symbols' closing price, and symbol as column name.
  Calculate symbols' performance in drawdown, Ulcer-Index, max-drawdown, standard deviation of returns,
  standard deviation of returns / Ulcer-Index, CAGR, CAGR / standard deviation of returns, CAGR / Ulcer-Index.

  '''

  from myUtils import symb_perf_stats_vectorized_v2
  # from trash import symb_perf_stats_vectorized_v2  

  symbols, period_yr, drawdown, UI, max_drawdown, returns_std, returns_std_div_UI, CAGR, CAGR_div_returns_std, CAGR_div_UI = \
      symb_perf_stats_vectorized_v2(df_close)

  caches_perf_stats_vect = []
  for symbol in symbols:
      # date_first = drawdown.index[0].strftime('%Y-%m-%d')
      # date_last = drawdown.index[-1].strftime('%Y-%m-%d')
      date_first = df_close.index[0].strftime('%Y-%m-%d')
      date_last = df_close.index[-1].strftime('%Y-%m-%d')

      cache = (symbol, date_first, date_last, period_yr, CAGR[symbol],
              UI[symbol], returns_std_div_UI[symbol], CAGR_div_returns_std[symbol], CAGR_div_UI[symbol])
      # append performance data (tuple) to caches_perf_stats (list)
      caches_perf_stats_vect.append(cache)

  column_names = ['symbol', 'first date', 'last date', 'Year', 'CAGR',
                  'UI', 'return_std/UI', 'CAGR/return_std', 'CAGR/UI']
  # write symbols' performance stats to dataframe
  df_perf = pd.DataFrame(caches_perf_stats_vect, columns=column_names)

  _cols = ['CAGR', 'UI', 'retrun_std/UI', 'CAGR/return_std', 'CAGR/UI']
  grp_CAGR_d_UI_mean = df_perf['CAGR/UI'].mean()
  grp_CAGR_d_UI_std  = df_perf['CAGR/UI'].std()
  grp_CAGR_d_UI_mean_div_std = grp_CAGR_d_UI_mean / grp_CAGR_d_UI_std 

  grp_CAGR_d_retnStd_mean = df_perf['CAGR/return_std'].mean()
  grp_CAGR_d_retnStd_std = df_perf['CAGR/return_std'].std()
  grp_CAGR_d_retnStd_mean_div_std = grp_CAGR_d_retnStd_mean / grp_CAGR_d_retnStd_std

  grp_retnStd_d_UI_mean = df_perf['return_std/UI'].mean()
  grp_retnStd_d_UI_std = df_perf['return_std/UI'].std()
  grp_retnStd_d_UI_mean_div_std = grp_retnStd_d_UI_mean / grp_retnStd_d_UI_std

  return df_perf, grp_retnStd_d_UI_mean_div_std, grp_CAGR_d_retnStd_mean_div_std, grp_CAGR_d_UI_mean_div_std


In [16]:
df_perf, grp_retnStd_d_UI_mean_div_std, grp_CAGR_d_retnStd_mean_div_std, grp_CAGR_d_UI_mean_div_std = perf_eval(df_c)
print(f'retnStd/UI   grp_mean/grp_std: {grp_retnStd_d_UI_mean_div_std:>10.6f}')
print(f'CAGR/retnStd grp_mean/grp_std: {grp_CAGR_d_retnStd_mean_div_std:>10.6f}')
print(f'CAGR/UI      grp_mean/grp_std: {grp_CAGR_d_UI_mean_div_std:>10.6f}')
df_perf.head()

retnStd/UI   grp_mean/grp_std:   2.015680
CAGR/retnStd grp_mean/grp_std:  -0.317811
CAGR/UI      grp_mean/grp_std:   0.051083


,symbol,first date,last date,Year,CAGR,UI,return_std/UI,CAGR/return_std,CAGR/UI
0,A,2021-12-21,2022-12-20,1.0,-0.033013,0.182462,0.121939,-1.483763,-0.180929
1,AA,2021-12-21,2022-12-20,1.0,-0.217883,0.402711,0.103447,-5.230113,-0.541040
2,AAL,2021-12-21,2022-12-20,1.0,-0.308499,0.252598,0.139765,-8.738256,-1.221305
3,AAON,2021-12-21,2022-12-20,1.0,-0.025720,0.266579,0.092280,-1.045539,-0.096483
4,AAP,2021-12-21,2022-12-20,1.0,-0.371006,0.216982,0.108161,-15.808331,-1.709850


In [14]:
my_symbols = ['AE', 'ABMD', 'RMBS', 'WNC']
my_cols = ['return_std/UI',	'CAGR/return_std', 'CAGR/UI']

In [19]:
%%timeit -n 50 -r 20
df_temp = df_c[my_symbols]
df_perf1, grp_StdUI_mean_div_std, grp_CAGRStd_mean_div_std, grp_CAGRUI_mean_div_std = perf_eval(df_temp)
# print(f'grp_Std/UI   mean/std: {grp_StdUI_mean_div_std:.6f}')
# print(f'grp_CAGR/Std mean/std: {grp_CAGRStd_mean_div_std:.6f}')
# print(f'grp_CAGR/UI  mean/std: {grp_CAGRUI_mean_div_std:.6f}')
# df_perf1

3.87 ms ± 425 µs per loop (mean ± std. dev. of 20 runs, 50 loops each)


In [20]:
%%timeit -n 50 -r 20
# select rows in my_symbols and columns in my_cols from df_perf
_df = df_perf.loc[df_perf['symbol'].isin(my_symbols)][my_cols]
grp_mean_div_grp_std = _df.mean() / _df.std()
# print(f'{my_cols[0]}   grp_mean/grp_std: {grp_mean_div_grp_std[0]:.6f}')
# print(f'{my_cols[1]} grp_mean/grp_std: {grp_mean_div_grp_std[1]:.6f}')
# print(f'{my_cols[2]}         grp_mean/grp_std: {grp_mean_div_grp_std[2]:.6f}')
# _df

2.3 ms ± 67.9 µs per loop (mean ± std. dev. of 20 runs, 50 loops each)


In [ ]:
df_SPY = df_c[['SPY']].copy()
df_SPY
df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std = perf_eval(df_SPY)
print(f'grp_StdUI_mean_std: {grp_StdUI_mean_std:.6f}')
print(f'grp_CAGRStd_mean_std: {grp_CAGRStd_mean_std:.6f}')
print(f'grp_CAGRUI_mean_std: {grp_CAGRUI_mean_std:.6f}')
df_perf.head()

In [ ]:
print(f'symbols:\n{symbols}')
print(f'period_yr:\n{period_yr}')
print(f'drawdown:\n{drawdown}')
print(f'UI:\n{UI}')
print(f'max_drawdown:\n{max_drawdown}')
print(f'returns_std:\n{returns_std}')
print(f'Std_UI:\n{Std_UI}')
print(f'CAGR:\n{CAGR}')
print(f'CAGR_Std:\n{CAGR_Std}')
print(f'CAGR_UI:\n{CAGR_UI}')